In [6]:
import sys
import numpy
import snappy
from snappy import ProductIO
from snappy import Product
from snappy import Band
from PIL import Image

In [7]:
jpy = snappy.jpy

# More Java type definitions required for image generation
Color = jpy.get_type('java.awt.Color')
ColorPoint = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef$Point')
ColorPaletteDef = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef')
ImageInfo = jpy.get_type('org.esa.snap.core.datamodel.ImageInfo')
ImageManager = jpy.get_type('org.esa.snap.core.image.ImageManager')
JAI = jpy.get_type('javax.media.jai.JAI')

# Disable JAI native MediaLib extensions 
System = jpy.get_type('java.lang.System')
System.setProperty('com.sun.media.jai.disableMediaLib', 'true')

'true'

In [8]:
file = '/home/jovyan/shared/eodata/Sentinel-2/MSI/L1C/2018/05/21/S2B_MSIL1C_20180521T100029_N0206_R122_T33UXT_20180521T120149.SAFE'
product = ProductIO.readProduct(file)
band = product.getBand('B2')
print(band)

org.esa.snap.core.datamodel.Band[B2,uint16,10980,10980,1,490.0,130.0,0.0]


In [9]:
def write_image(band, points, filename, format):
    cpd = ColorPaletteDef(points)
    ii = ImageInfo(cpd)
    band.setImageInfo(ii)
    im = ImageManager.getInstance().createColoredBandImage([band], band.getImageInfo(), 0)
    JAI.create("filestore", im, filename, format)

In [10]:
points = [ColorPoint(0.05, Color.YELLOW), 
          ColorPoint(0.10, Color.RED), 
          ColorPoint(0.15, Color.BLUE)]

write_image(band, points, 'snappy_colored_band.png', 'PNG')

In [11]:
im = Image.open("snappy_colored_band.png")
nx, ny = im.size
im2 = im.resize((int(nx*0.2), int(ny*0.2)), Image.BICUBIC)
im2.save("snappy_colored_band_res.png")

/opt/conda/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [12]:
layer=ImageLayer()
box=layer.getbb(file)
layer.addImageLayer('snappy_colored_band_res.png', box, 'S2_B2_color')
layer.showLayer()